<a href="https://colab.research.google.com/github/Abhishekmystic-KS/ai-image-restoration/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pip install tensorflow tensorflow-hub numpy Pillow

In [7]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
from PIL import Image
import os

In [6]:
# 1. Load the model once
print("Loading ESRGAN model...")
model = hub.load('https://tfhub.dev/captain-pool/esrgan-tf2/1')

def restore_image(image_path, output_path):
    # Load Image
    img = Image.open(image_path).convert('RGB')

    # --- RAM OPTIMIZATION: Resize if the image is too big ---
    # ESRGAN 4x will crash if the input is > 1000px.
    # We downsize the input so the output stays under control.
    max_size = 600
    if max(img.size) > max_size:
        print(f"Resizing input from {img.size} to save RAM...")
        img.thumbnail((max_size, max_size), Image.LANCZOS)

    # Convert to Tensor (Removed / 255.0 normalization as model expects 0-255 range)
    img_tensor = tf.expand_dims(np.array(img).astype(np.float32), axis=0)

    print("Processing... (This uses high RAM/GPU)")
    # Run the model
    restored_tensor = model(img_tensor)

    # Convert back to PIL (Removed * 255.0 and added clipping for 0-255 range)
    restored_tensor = tf.squeeze(restored_tensor, axis=0)
    restored_img_array = tf.cast(tf.clip_by_value(restored_tensor, 0, 255), tf.uint8).numpy()
    restored_image = Image.fromarray(restored_img_array)

    # Save result
    restored_image.save(output_path)
    print(f"Done! Restored image saved as: {output_path}")

# Run the function
# Change '1000106629.jpg' to your actual filename or upload '1000106629.jpg' to your Colab environment
restore_image('1000106629.jpg', 'restored_output.png')

Loading ESRGAN model...
Resizing input from (1280, 960) to save RAM...
Processing... (This uses high RAM/GPU)
Done! Restored image saved as: restored_output.png
